Vagrantプロビジョニングで設定できない項目の初期設定

# 前処理

## インポート

In [ ]:
from fabric import Connection, Config
import subprocess, re, time

## 関数 - ping応答待ち(Windows)

In [ ]:
def wait_ping(ip, timeout=60):
    for _ in range(timeout):
        if subprocess.run(["ping", "-n", "1", "-w", "1000", ip], stdout=subprocess.DEVNULL).returncode == 0:
            print('OK')
            return
        time.sleep(1)
    raise TimeoutError(f"{ip} unreachable")

## 接続情報
IPアドレスはDefault Switchで割り当てられたDHCPアドレスを取得

In [ ]:
result = subprocess.run(["vagrant", "ssh-config", "slurm-master"], capture_output=True, text=True)
match = re.search(r'^\s*HostName\s+(.+)$', result.stdout, re.MULTILINE)
remote_host = match.group(1)
print(remote_host)

In [ ]:
remote_user = 'vagrant'
key_file    = '.vagrant/machines/slurm-master/hyperv/private_key'
new_address = '192.168.128.10'

## fabric設定

In [ ]:
config = Config(overrides={
    'run': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': False,'hide': False,'timeout': 20},
    'sudo': {'env': {'LANG': 'C'},'echo': True,'warn': True,'pty': True,'hide': False,'timeout': 20,'password': None},
    'timeouts': {'connect': 5,'command': 30,},
})

# 接続

## SSH接続

In [ ]:
c = Connection(
    port=22,
    host=remote_host,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

# 処理実行

## Hyper-V内部ネットワークのIPアドレス設定
プロビジョニング完了のタイミングでNICを割り当てしているためこのタイミングで設定

In [ ]:
c.run('ip a')

In [ ]:
c.sudo(f"""bash -c 'cat > /etc/netplan/99-hyperv-internal.yaml <<EOF
network:
  version: 2
  ethernets:
    eth1:
      dhcp4: false
      addresses:
        - {new_address}/24
EOF
'""")

In [ ]:
c.sudo("netplan apply", pty=False, warn=True, disown=True)

In [ ]:
wait_ping(new_address)

In [ ]:
c.close()
c = Connection(
    port=22,
    host=new_address,
    user=remote_user,
    connect_kwargs={
        'key_filename': key_file,
    },
    config=config,
)
c.run('hostname;date;whoami')

In [ ]:
c.run('sudo ip a')

## munge.key ダウンロード
各ノードへの配布用

In [ ]:
c.sudo('cp /etc/munge/munge.key /home/vagrant/')

In [ ]:
c.sudo('chmod 644 /home/vagrant/munge.key')

In [ ]:
c.run('ls -l /home/vagrant')

In [ ]:
c.get('/home/vagrant/munge.key', 'munge.key')

In [ ]:
c.sudo('rm -f /home/vagrant/munge.key')

## SSH鍵作成、ダウンロード
ノードへのSSH接続用

In [ ]:
c.run('ssh-keygen -t rsa -f ~/.ssh/id_rsa -q -N ""')

In [ ]:
c.run('cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys')

In [ ]:
c.get(".ssh/id_rsa.pub", "id_rsa.pub")

# 切断

In [ ]:
c.close()